# Register iNat observations in openBIS using pyBIS
Demo notebook to import iNat observation in CSV file into openBIS. The CSV file is output generated by `inat_fetcher.py`.

In [1]:
import pandas as pd
import getpass
from pybis import Openbis

### 1. Connect to openBIS

In [2]:
o = Openbis('https://XYZ.labnotebook.ch/', verify_certificates=False)

In [ ]:
if o.is_session_active():
    print(f"Session is active and token is {o.token}")
else:
    password = getpass.getpass()
    o.login('hluetcke_admin', password, save_token=True)   # save the session token in ~/.pybis/example.com.token
    del password
    print(f"Session is active: {o.is_session_active()} and token is {o.token}")

### 2. Read CSV file

In [4]:
csv_file = '../data/out/test_inat_output_hl.csv'

df = pd.read_csv(csv_file)

properties_csv = df.columns.tolist()

### 3. Register new samples
Todo: option for updating existing samples

In [5]:
sample_type = 'OBSERVATION'
space = 'MATERIALS'
collection = '/MATERIALS/SAMPLES/SAMPLES_EXP_4'

In [6]:
samples = o.get_samples(type=sample_type, collection=collection, props='observation_id')

In [7]:
def get_openbis_properties(row, properties):
    # get properties for openBIS sample
    properties_ob = {}
    for prop in properties:
        if prop == 'id':
            prop_ob = 'observation_id'
        else:
            prop_ob = prop
            
        properties_ob[prop_ob] = row[prop]

    return properties_ob

In [ ]:
break_tf = False
for index, row in df.iterrows():
    print(row['id'])
    tf = samples.df['OBSERVATION_ID'] == str(row['id'])
    if tf.any():
        print('sample has been registered')
        # option to update sample metadata
    else:
        print('sample has not been registered yet. registering now ...')

        properties_ob = get_openbis_properties(row, properties_csv)

        sample = o.new_sample(
            type       = sample_type,
            space      = space,
            experiment = collection,
            props      = properties_ob
        )
        sample.save()
        print('sample has been registered with code ' + sample.code)
        break_tf = True
    
    if break_tf:
        break

KeyError: 'No such property: «observed_on_details.date». Allowed properties are: observation_id, quality_grade, time_observed_at, taxon_geoprivacy, annotations, uuid, cached_votes_total, identifications_most_agree, species_guess, identifications_most_disagree, tags, positional_accuracy, comments_count, site_id, license_code, quality_metrics, public_positional_accuracy, reviewed_by, oauth_application_id, flags, created_at, description, project_ids_with_curator_id, updated_at, sounds, place_ids, captive, ident_taxon_ids, outlinks, faves_count, ofvs, num_identification_agreements, comments, map_scale, uri, project_ids, community_taxon_id, owners_identification_from_vision, identifications_count, obscured, num_identification_disagreements, geoprivacy, location, votes, spam, mappable, identifications_some_agree, project_ids_without_curator_id, place_guess, identifications, project_observations, photos, faves, non_owner_ids, observed_on, photo_url, taxon_is_active, taxon_ancestry, taxon_min_species_ancestry, taxon_endemic, taxon_iconic_taxon_id, taxon_min_species_taxon_id, taxon_threatened, taxon_rank_level, taxon_introduced, taxon_native, taxon_parent_id, taxon_name, taxon_rank, taxon_extinct, taxon_id, taxon_ancestor_ids, taxon_photos_locked, taxon_taxon_schemes_count, taxon_wikipedia_url, taxon_current_synonymous_taxon_ids, taxon_created_at, taxon_taxon_changes_count, taxon_complete_species_count, taxon_universal_search_rank, taxon_observations_count, taxon_flag_counts_resolved, taxon_flag_counts_unresolved, taxon_atlas_id, taxon_default_photo_id, taxon_default_photo_license_code, taxon_default_photo_attribution, taxon_default_photo_url, taxon_default_photo_original_dimensions_height, taxon_default_photo_original_dimensions_width, taxon_default_photo_flags, taxon_default_photo_square_url, taxon_default_photo_medium_url, taxon_iconic_taxon_name, taxon_preferred_common_name, preferences_prefers_community_taxon, geojson_coordinates, geojson_type, user_site_id, user_created_at, user_id, user_login, user_spam, user_suspended, user_login_autocomplete, user_login_exact, user_name, user_name_autocomplete, user_orcid, user_icon, user_observations_count, user_identifications_count, user_journal_posts_count, user_activity_count, user_species_count, user_universal_search_rank, user_roles, user_icon_url, taxon_default_photo, taxon_conservation_status_place_id, taxon_conservation_status_source_id, taxon_conservation_status_user_id, taxon_conservation_status_authority, taxon_conservation_status_status, taxon_conservation_status_status_name, taxon_conservation_status_geoprivacy, taxon_conservation_status_iucn, observed_on_details, created_time_zone, observed_time_zone, time_zone_offset, observed_on_string, created_at_details_date, created_at_details_week, created_at_details_month, created_at_details_hour, created_at_details_year, created_at_details_day, swiped_loc, emi_external_id'

In [9]:
properties_ob

{'observation_id': 112079127,
 'quality_grade': 'casual',
 'taxon_geoprivacy': 'open',
 'annotations': '[]',
 'uuid': '4fba04ec-053e-46c2-8bf4-1db7258410c5',
 'observed_on_details.date': '2022-04-19',
 'observed_on_details.day': 19,
 'observed_on_details.month': 4,
 'observed_on_details.year': 2022,
 'observed_on_details.hour': 11,
 'observed_on_details.week': 16,
 'cached_votes_total': 0,
 'identifications_most_agree': False,
 'created_at_details.date': '2022-04-19',
 'created_at_details.day': 19,
 'created_at_details.month': 4,
 'created_at_details.year': 2022,
 'created_at_details.hour': 22,
 'created_at_details.week': 16,
 'species_guess': 'Avocatier',
 'identifications_most_disagree': False,
 'tags': '[]',
 'positional_accuracy': nan,
 'comments_count': 0,
 'site_id': 1,
 'created_time_zone': 'Europe/Paris',
 'license_code': 'cc0',
 'observed_time_zone': 'Europe/Paris',
 'quality_metrics': "[{'id': 9076457, 'user_id': 5367650, 'metric': 'wild', 'agree': False, 'user': {'id': 53676